In [7]:
import geopandas as gpd
import os
import fiona
import ipywidgets as widgets
from IPython.display import display
from rasterstats import zonal_stats
import rasterio
import rasterio.fill
from shapely.geometry import shape, mapping
import json
from earthpy import clip
import earthpy.spatial as es
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import datetime
import warnings
import pandas as pd
import scipy.spatial
from pandas import DataFrame as df
from shapely.geometry import Point, Polygon
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

#  Performing spatial join (merge) using geopandas

## CROP1 :MAIZE

In [8]:
path = r"C:\Benin\agrodem_output"
name_of_file= "test.csv"
# Import csv as pandas dataframe
df_maize= pd.read_csv(path + "\\" + name_of_file)

In [9]:
new = df(df_maize,columns =['Pixel', 'State', 'lon', 'lat', 'Crop','Annual_elec_demand'])

In [10]:
new.rename(columns={"State":"name_adm2"}, inplace=True)

In [11]:
new.head()

,Pixel,name_adm2,lon,lat,Crop,Annual_elec_demand
0,NaN,Karimama,2.91256,12.29472,Maize,47009.709570
1,NaN,Karimama,2.93052,12.28589,Maize,28253.623800
2,NaN,Karimama,2.97544,12.28589,Maize,3018.444945
3,NaN,Karimama,2.99340,12.26822,Maize,3947.988300
4,NaN,Karimama,3.16408,12.07379,Maize,31341.849310


In [12]:
# Add geometry and convert to spatial dataframe in source CRS
new['geometry'] = list(zip(new['lon'], new['lat']))
new['geometry'] = new['geometry'].apply(Point)
new = gpd.GeoDataFrame(new, geometry='geometry', crs={'init': 'epsg:4326'})

# Reproject data in to Ordnance Survey GB coordinates
new_osgb = new.to_crs({'init': 'epsg:3395'})

In [13]:
# save
path = r"C:\Benin\agrodem_output_postprocessing"
#csv:
shpname = "crop_irrigation_demand.csv"
new_osgb.to_csv(os.path.join(path,"{c}".format(c=shpname)), index=False)
#shp:
new_osgb.to_file('crop_irrigation_demand.shp',driver = 'ESRI Shapefile')
# Define output path

In [82]:
path = r"C:\Benin\agrodem_output"
name_of_file= "yams_filtered_results_1km.csv"
# Import csv as pandas dataframe
df_yam= pd.read_csv(path + "\\" + name_of_file)

In [83]:
new_yam = df(df_yam,columns =['Pixel', 'State', 'lon', 'lat', 'Crop','Annual_elec_demand'])

In [84]:
new_yam.rename(columns={"State":"name_adm2"}, inplace=True)

In [85]:
# Add geometry and convert to spatial dataframe in source CRS
new_yam['geometry'] = list(zip(new_yam['lon'], new_yam['lat']))
new_yam['geometry'] = new_yam['geometry'].apply(Point)
new_yam = gpd.GeoDataFrame(new_yam, geometry='geometry', crs={'init': 'epsg:4326'})

# Reproject data in to Ordnance Survey GB coordinates
new_yam_osgb = new_yam.to_crs({'init': 'epsg:3395'})

In [86]:
# save
path = r"C:\Benin\agrodem_output_postprocessing"
#csv:
shpname = "yam_irrigation_demand.csv"
new_yam_osgb.to_csv(os.path.join(path,"{c}".format(c=shpname)), index=False)
#shp:
new_yam_osgb.to_file('yam_irrigation_demand.shp',driver = 'ESRI Shapefile')
# Define output path

### Import admin 2 layer


In [59]:
path = r"C:\Oluchi\Benin\Admin_boundaries_modified\Admin_boundaries_modified_3395"
name_shp = "ben_admin_boundaries_modified_3395.shp"
admin_2_gdf = gpd.read_file(path + "\\" + name_shp)

### Import previously exported shapefile

In [60]:
path = r"C:\Github\agrodem_new"
name_shp = "maize_irrigation_demand.shp"
maize_shp = gpd.read_file(path + "\\" + name_shp)

In [54]:
#admin_2_maize_gdf = gpd.sjoin(maize_shp,admin_2_gdf, how = "right", op = "intersects")

In [66]:
admin_2_maize_gdf.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,...,CC_2,HASC_2,name_adm2,geometry_x,Pixel,lon,lat,Crop,Annual_ele,geometry_y
0,BEN,Benin,BEN.1_1,Alibori,None,BEN.1.4_1,Karimama,None,None,Commune,...,None,BJ.AL.KR,Karimama,"POLYGON ((366443.493 1328774.638, 365336.536 1...",11142,2.91256,12.29472,Maize,40526089.54,POINT (324224.696 1370174.961)
1,BEN,Benin,BEN.1_1,Alibori,None,BEN.1.4_1,Karimama,None,None,Commune,...,None,BJ.AL.KR,Karimama,"POLYGON ((366443.493 1328774.638, 365336.536 1...",11169,2.93052,12.28589,Maize,29088330.00,POINT (326223.994 1369175.385)
2,BEN,Benin,BEN.1_1,Alibori,None,BEN.1.4_1,Karimama,None,None,Commune,...,None,BJ.AL.KR,Karimama,"POLYGON ((366443.493 1328774.638, 365336.536 1...",11174,2.97544,12.28589,Maize,19841346.03,POINT (331224.466 1369175.385)
3,BEN,Benin,BEN.1_1,Alibori,None,BEN.1.4_1,Karimama,None,None,Commune,...,None,BJ.AL.KR,Karimama,"POLYGON ((366443.493 1328774.638, 365336.536 1...",11249,2.99340,12.26822,Maize,31710326.40,POINT (333223.764 1367175.204)
4,BEN,Benin,BEN.1_1,Alibori,None,BEN.1.4_1,Karimama,None,None,Commune,...,None,BJ.AL.KR,Karimama,"POLYGON ((366443.493 1328774.638, 365336.536 1...",12018,3.16408,12.07379,Maize,49433072.38,POINT (352223.774 1345175.339)


In [ ]:
# save
path = r"C:\Benin\agrodem_output_postprocessing"
#csv:
shpname = "maize_irrigation_demand.csv"
new_osgb.to_csv(os.path.join(path,"{c}".format(c=shpname)), index=False)
#shp:
new_osgb.to_file('maize_irrigation_demand.shp',driver = 'ESRI Shapefile')
# Define output path

# NEW ONE

In [139]:

# import the two csv files needed
#STEP1 : Import the csv file containing the hubdistance
path = r"C:\Benin\agrodem_output_postprocessing"
name_of_file = "hub_distance.csv"
# Import csv as pandas dataframe
distance_file= pd.read_csv(path + "\\" + name_of_file)
distance_file.head(3)

,Pixel,name_adm2,lon,lat,Crop,Annual_ele,HubName,HubDist
0,NaN,Bembereke,3.10560,10.54367,Yams,4360.912086,32.221730,8081.607901
1,NaN,Bembereke,2.54864,10.53478,Yams,5489.706550,17.079075,180.907603
2,NaN,Bembereke,2.56661,10.53478,Yams,546.614767,6.275945,1446.012351


In [140]:
distance_file.rename(columns={'HubName':'Pop'}, inplace=True)

In [141]:
distance_file.columns

Index(['Pixel', 'name_adm2', 'lon', 'lat', 'Crop', 'Annual_ele', 'Pop',
       'HubDist'],
      dtype='object')

In [142]:
distance_file= df(distance_file,columns =['Annual_ele', 'Pop'])

In [143]:
distance_file.describe()

,Annual_ele,Pop
count,6.460000e+02,648.000000
mean,2.159522e+07,81041.093346
std,2.330422e+07,132989.040486
min,4.168971e+02,6.102459
25%,1.027872e+04,14.683779
50%,1.988444e+07,43.384276
75%,3.921691e+07,299504.088837
max,1.169775e+08,299504.088837


In [144]:
#sort by the population
distance_file=distance_file.groupby('Pop').sum()

In [145]:
distance_file.head()

,Annual_ele
Pop,
6.102459,9.738180e+08
6.275945,5.466148e+02
7.341890,7.448236e+07
8.375394,1.125304e+04
8.539538,9.397619e+07


In [150]:
path = r"C:\Benin\New folder"

distance_file.to_csv(os.path.join(path,"{c}.csv".format(c="hub_final")))

In [146]:
# import the two csv files needed
#STEP1 : Import the csv file containing the hubdistance
path = r"C:\Benin\agrodem_output_postprocessing"
name_of_file = "Benin_calibrated.csv.csv"
# Import csv as pandas dataframe
input_file= pd.read_csv(path + "\\" + name_of_file)
input_file.head(3)

,id,GridCellArea,Country,Pop,NightLights,ElecPop,DHSREGEN,WindVel,GHI,TravelHours,...,PopStartYear,ElecPopCalib,Pop2025High,Pop2025Low,Pop2030High,Pop2030Low,Pop2020,ElecStart,GridDistCalibElec,FinalElecCode2020
0,62054,0.507376,Benin,3713.732478,3.208329,3713.732478,Mono,5.988861,1899.927076,0.642423,...,3716.936013,3516.651898,3978.143990,3978.143990,4257.708378,4257.708378,3716.936013,1,0.0,1
1,62053,0.180592,Benin,1096.123683,0.815084,1096.123683,Mono,5.905779,1898.280151,0.953415,...,1097.069220,1037.954525,1174.165847,1174.165847,1256.680446,1256.680446,1097.069220,1,0.0,1
2,62052,0.008600,Benin,65.440220,1.399128,65.440220,Mono,5.879070,1898.000000,0.900000,...,65.496670,61.967434,70.099454,70.099454,75.025698,75.025698,65.496670,1,0.0,1


In [147]:
new_input_final= input_file.merge(distance_file,on ='Pop', how = 'left')

In [148]:
x=new_input_final['Annual_ele'].sum()
print(x)

0.0


In [ ]:
messagebox.showinfo('OnSSET extraction', 'Output folder')
workspace = filedialog.askdirectory()

messagebox.showinfo('OnSSET', 'Select the admin boundaries')
admin = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

## Select the clusters

In [ ]:
crs = 'EPSG:3395'

In [ ]:
messagebox.showinfo('OnSSET', 'Select the clusters')
clusters = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    
popunit = widgets.Dropdown(options=clusters.head(),
    value=None,
    description='Population:',
    disabled=False)
display(popunit)
x = popunit.value

In [ ]:
def preparing_for_vectors(workspace, clusters, crs):   
    clusters.crs = {'init' :'epsg:4326'}
    clusters = clusters.to_crs({ 'init': crs}) 
    points = clusters.copy()
    points["geometry"] = points["geometry"].centroid
    points.to_file(workspace + r'\clusters_cp.shp', driver='ESRI Shapefile')
    print(datetime.datetime.now())    
    return clusters

In [ ]:
clusters = preparing_for_vectors(workspace, clusters, crs)

In [ ]:
messagebox.showinfo('OnSSET', 'Select the irrigation demand map')
irrdem=gpd.read_file(filedialog.askopenfilename(title = "Select Irrigation Demand map", filetypes = (("shapefile","*.shp"),("all files","*.*"))))

irrdemand = widgets.Dropdown(options=irrdem.head(),
    value=None,
    description='IrrigationDemand:',
    disabled=False)

display(irrdemand)
      
IrrDemandunit = widgets.Dropdown(options=['Wh', 'kWh', 'MWh'],
    value='kWh',
    description='Unit:',
    disabled=False)

display(IrrDemandunit)